In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
import random

from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import text, sequence
from keras.layers import Embedding, SpatialDropout1D
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.optimizers import RMSprop
import keras.backend as K
from keras.layers import Dense, Input, GRU, LSTM, Bidirectional, Dropout, CuDNNLSTM, CuDNNGRU
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [3]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#2. Get the file
data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

In [0]:
train['mal'] = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [0]:
#X = train.comment_text
#y = train.mal

max_features = 40000
maxlen = 400
dropout_rate = 0
rs = 42
epochs = 4
batch_size = 250
embed_dim = 50
rec_units = 150

In [0]:
def gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units):
    if K.backend == 'tensorflow':        
        K.clear_session()
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(max_features, output_dim=embed_dim, trainable=True)(input_layer)
    x = SpatialDropout1D(dropout_rate)(embedding_layer)
    x = Bidirectional(CuDNNGRU(units=rec_units, return_sequences=False))(x)
    output_layer = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
                  metrics=['acc'])
    #print( model.summary())
    return model

In [0]:
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0

In [11]:
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    tokenizer = text.Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(X_train)
    list_tokenized_train = tokenizer.texts_to_sequences(X_train)
    list_tokenized_val   = tokenizer.texts_to_sequences(X_val)
    X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_val                = sequence.pad_sequences(list_tokenized_val, maxlen=maxlen)
    model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=True, verbose=1)
    model.save_weights(f'{cv_models_path}/BGRU_fold_{c}.h5')
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {round(auc_f, 3)}')
    print(f' roc auc {round(roc_f, 3)}')
    c += 1
    del model
    K.clear_session()

 fold 0
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 138s 1ms/step - loss: 0.1790 - acc: 0.9370 - val_loss: 0.1165 - val_acc: 0.9608
Epoch 2/4
126974/126974 [==============================] - 135s 1ms/step - loss: 0.1198 - acc: 0.9538 - val_loss: 0.1291 - val_acc: 0.9592
Epoch 3/4
126974/126974 [==============================] - 135s 1ms/step - loss: 0.1098 - acc: 0.9568 - val_loss: 0.1072 - val_acc: 0.9603
Epoch 4/4
31745/31745 [==============================] - 10s 315us/step
 average precision 0.89
 roc auc 0.977
 fold 1
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 135s 1ms/step - loss: 0.1898 - acc: 0.9335 - val_loss: 0.1473 - val_acc: 0.9534
Epoch 2/4
126974/126974 [==============================] - 135s 1ms/step - loss: 0.1197 - acc: 0.9536 - val_loss: 0.1100 - val_acc: 0.9609
Epoch 3/4
126974/126974 [==============================] - 1

In [0]:
data = pd.DataFrame({'acc':history.history['acc'],
                    'loss': history.history['loss'],
                    'val_acc': history.history['val_acc'],
                    'val_loss': history.history['val_loss']})

In [13]:
data

,acc,loss,val_acc,val_loss
0,0.938768,0.174939,0.933308,0.164459
1,0.959607,0.109357,0.921305,0.179449
2,0.962749,0.097854,0.923227,0.174087
3,0.964363,0.092763,0.938758,0.142677


In [14]:
round(np.array(auc).mean(), 3)

0.861

In [15]:
round(np.array(roc).mean(), 3)

0.973

In [22]:
K.clear_session()
X_train = X
y_train = y

tokenizer            = text.Tokenizer(num_words=max_features, oov_token='unknown')
tokenizer.fit_on_texts(X_train)

list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test  = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

model = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)

y_train              = np.array(y_train)
y_test               = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
model.save_weights(f'{models_path}/BGRU.h5')
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)

Fitting
Epoch 1/4
158719/158719 [==============================] - 157s 989us/step - loss: 0.1685 - acc: 0.9400
Epoch 2/4
158719/158719 [==============================] - 157s 989us/step - loss: 0.1129 - acc: 0.9560
Epoch 3/4
158719/158719 [==============================] - 156s 985us/step - loss: 0.1062 - acc: 0.9580
Epoch 4/4
64830/64830 [==============================] - 20s 313us/step


In [23]:
round(auc_f, 3)

0.86

In [24]:
round(roc_f, 3)

0.973